## 3rdラボへようこそ -  第1週、4日目

今日は、すぐに価値のあるものを構築します！

フォルダー `me` に、単一のファイル `linkedIn.pdf` を配置しました。これは、linkedInプロファイルのpdfダウンロードです。

あなたのものに置き換えてください！

`summary.txt`というファイルも作成しました

まだツールを使用するつもりはありません。明日ツールを追加します。

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">パッケージの検索</h2>
            <span style="color:#00bfff;">このラボでは、簡単なUIを構築するための優れたGradioパッケージと、人気のPDFリーダーPyPDFを使用します。これらのパッケージのガイドは、ChatGPTまたはClaudeに問い合わせることで入手できます。また、すべてのOSSパッケージはリポジトリ<a href="https://pypi.org">https://pypi.org</a>で見つけることができます。
            </span>
        </td>
    </tr>
</table>

In [1]:
!pip install pypdf

In [2]:
# これらのパッケージが何をしているのかわからない場合は、いつでもChatGPTにガイドを尋ねることができます！

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [3]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [8]:
print(linkedin[:100]+"...")

   
Contact
ed.donner@gmail.com
www.linkedin.com/in/eddonner
(LinkedIn)
edwarddonner.com (Personal)
...


In [9]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [10]:
name = "Ed Donner"

In [11]:
#「あなたは{name}として行動します。{name}のウェブサイト上の質問、特に{name}のキャリア、経歴、スキル、経験に関する質問に回答します。\
# あなたの責任は、ウェブサイト上でのやり取りにおいて、{name}をできる限り忠実に代表することです。質問への回答には、{name}の経歴とLinkedInプロフィールの概要が提供されます。
# ウェブサイトを訪れた潜在的な顧客や将来の雇用主に話しかけるかのように、プロフェッショナルで魅力的な対応を心がけてください。答えがわからない場合は、その旨を伝えてください。
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

# ## 概要:{summary} ## LinkedIn プロフィール:{linkedin}"
system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
# このコンテキストでは、常に {name} としてのキャラクターを維持しながらユーザーとチャットしてください。
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## OpenAIを使用していない人々のための特別なメモ

Groqのような一部のプロバイダーは、チャットで2番目のメッセージを送信するときにエラーを発生させる可能性があります。

これは、Gradioが履歴オブジェクトにいくつかの余分なフィールドを押し込むためです。 OpenAIは気にしません。しかし、他のいくつかのモデルは不平を言っています。

これが発生した場合、解決策は、この最初の行を上記のChat()関数に追加することです。履歴変数をクリーンアップします：

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

これを他のChat()関数のコールバック機能に将来的に追加する必要がある場合があります。

In [13]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## 多くのことが起こりそうです...

1. LLMに回答の評価を依頼できる
2. 回答が評価に失敗した場合に再実行できる
3. これらを1つのワークフローにまとめる

すべてエージェントフレームワークなし！

In [ ]:
# 評価のためにPydanticモデルを作成
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [14]:
# あなたは、質問への回答が適切かどうかを判断する評価者です。ユーザーとエージェントとの会話が表示されます。
# あなたの仕事は、エージェントの最新の回答が適切かどうかを判断することです。エージェントは{name}の役割を演じ、ウェブサイト上で{name}を代表しています。
# エージェントは、ウェブサイトにアクセスした潜在的な顧客や将来の雇用主に話しかけるかのように、プロフェッショナルで魅力的な対応をするように指示されています。
# エージェントには、{name}の概要とLinkedInの詳細という形で、{name}に関するコンテキストが提供されています。情報は次のとおりです。
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

# ## 概要:{summary} ## LinkedIn プロフィール:{linkedin}"
evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
# この文脈を踏まえて、最新の応答を評価し、応答が受け入れ可能かどうかとフィードバックを返信してください。
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
# ユーザーとエージェントの会話は次のとおりです: {history}
# ユーザーからの最新のメッセージは次のとおりです: {message}
# エージェントからの最新の応答は次のとおりです: {reply}
# 応答を評価し、適切かどうか、そしてフィードバックを返信してください。
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "do you hold a patent?", messages[:1])

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n# ＃以前の回答は拒否された\ nyouを返信しようとしましたが、品質管理はあなたの返信を拒否しました\ n "
    updated_system_prompt += f"# ＃あなたの試みられた答え：\ n {Reply} \ n \ n "
    updated_system_prompt += f"# ＃拒絶の理由：\ n {フィードバック} \ n \ n "
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("渡された評価 - 返信を返します")
    else:
        print("評価の失敗 - 再試行")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()